In [1]:
# import pymongo and connect Mong client with the ipadress from the shell
# fix IssueBook mthod issue  Negative --> Positive --> Issue
import pymongo
import pandas as pd
from datetime import datetime
from datetime import timedelta
mongo=pymongo.MongoClient("mongodb://127.0.0.1:27017/")
#bookCollection=mongo['library']['books']
#studentCollection=mongo['library']['student']
#penaltyCollection=mongo['library']['penalty']
#bookCollection.drop()
#studentCollection.drop()
#spenaltyCollection.drop()

In [2]:
class library:
    def __init__(self):
        self.bookCollection=mongo['library']['books']
    
    # Method to execute Main Menu to choose between Admin profile and Student Profile. 
    def mainMenu(self):
        self.choice = input('Enter 1 (if you are an Admin) or 2 (if you are Student):')
        if self.choice.isdigit() and (int(self.choice) == 1 or int(self.choice) == 2):
            print('\nHey Hi!! Welcome to the Library Management System')
        else:
            print('You have enter Invalid Option. Pls retry!\n')
            self.mainMenu()
            
        return int(self.choice)
    
    # This method will display list of all Books available in collections. 
    def allBooksList(self):
        print('\n**************List of Books in Library**************\n')
        self.bookCollection=mongo['library']['books']
        self.bookList=self.bookCollection.find({},{'issue_date':0,'renew_date':0,'assigned_to':0,'penalty':0})
        self.booksdf=pd.DataFrame(list(self.bookList))
        print(self.booksdf)
        
        self.final=self.thankYou()
        return self.final
        
    # This methode will display list of all Books available to Issue in collections. 
    def availableBooksList(self):
        print('\n**************List of Availble Books in Library**************\n')
        self.bookCollection=mongo['library']['books']
        self.availableBookList=self.bookCollection.find({'assigned_to':None},{'issue_date':0,'renew_date':0,'assigned_to':0,'penalty':0})
        self.availableBooksdf=pd.DataFrame(list(self.availableBookList))
        print(self.availableBooksdf)   
                
        self.final=self.thankYou()
        return self.final
        
    # This method calcualates the number of days between current date and Renew Date. . 
    def renewDateValidation(self,renewDate):
        self.currentDate = str(datetime.now())
        self.currentDate=self.currentDate[0:10]
        self.d1 = datetime.strptime(renewDate, "%Y-%m-%d")
        self.d2 = datetime.strptime(self.currentDate, "%Y-%m-%d")
        self.daysDifference=((self.d2 - self.d1).days)
        return self.daysDifference 
    
    # This method will calculate Penalty, if applicable and update in the collection ===> Dilip
    def updatePenalty(self):
        self.bookCollection=mongo['library']['books']
        self.L1=[]
        
        self.upList=self.bookCollection.find({'assigned_to':{'$ne':None}},{'_id':1}) #fetching ISBN of Books issues
        for self.data in self.upList:
            self.L1.append(self.data['_id'])
            
        for self.ISBN in self.L1:
            self.getData=self.bookCollection.find_one({'_id':self.ISBN},{})          # Fetching data for each ISBN
            self.daysGap = self.renewDateValidation(self.getData['renew_date'])      # Calculating the Penalty Amount based on Renew Date   
            if self.daysGap  > 0:
                self.uPenalty = self.daysGap * 20
                self.bookCollection.update_one({'_id':self.getData['_id']},{'$set':{'penalty':self.uPenalty}})  # Updating the collections

    # This method is the ThankYou method .. 
    def thankYou(self):
        print('\n********Thank you for using Library Management System********.\nPls enter Y to restart from beginning or N to Exit')
        finalChoice=input()
        return finalChoice

#  This class is Purely for ADMIN Rights. 
class admin(library):
    def __init__(self):
        self.bookCollection=mongo['library']['books']
        print('Hello Admin. Pls login to proceed\n')

    # This method is accepting Login and Password.             
    def adminLogin(self):
        self.userID = input('Enter the Login ID :')
        self.userPWD = input('Enter the Password :')
        if self.userID == 'admin' and self.userPWD == 'admin':
            print('\nCongrats, you have logged in succesfully as Admin')
            self.adminMenuChoice=self.adminMenu()
        else:
            print('Invalid User ID or Password. Pls try again')
            self.adminLogin()
        
        return self.adminMenuChoice
    
    # This particular method is used to display the Admin Menu option and accepts Admin input from the Menu Options
    def adminMenu(self):
        print('\nChoose the Option Menu Option\n*************************************!')
        print('Enter 1 to ADD New book into Repository.')
        print('Enter 2 to EDIT existing book in Repository.')
        print('Enter 3 to DELETE book from Repository.')
        print('Enter 4 to ISSUE the book to Student.')
        print('Enter 5 to RECEIVE the book.')
        print('Enter 6 to RENEW the book.')
        print('Enter 7 to view ISSUED Books.')
        print('Enter 8 to view list of AVAILABLE Books.')
        print('Enter 9 to view list of ALL Books.')
        print('Enter 10 to accept PENALTY.')
        self.adminChoice=input()
        if self.adminChoice.isdigit() == False or int(self.adminChoice) < 1 or int(self.adminChoice) > 10:
            print('Invalid Selection. Pls try again!!')
            self.adminMenu()
        else:
            self.adminChoice = int(self.adminChoice)
        
        return self.adminChoice
    
    # This method is used to add the new book to the Library
    def addBook(self):
        self.bookCollection=mongo['library']['books']
        
        self.ISBN= input('\nEnter the ISBN of the Book  :')
        self.bookCheck=self.bookCollection.find_one({'_id':int(self.ISBN)},{})

        # Validating the ISBN, if already present in Library
        if self.bookCheck != None or self.ISBN.isdigit() == False or len(self.ISBN) != 5:
            print(f'Book with ISBN {self.ISBN} is either non-numberic of 5 digits or ISBN {self.ISBN} is already in database.\nPls Retry with another Book with 5 digits Numeric ISBN. ')
            self.addBook()
            
        self.Title= input('Enter the Book Title        :')
        self.Author=input('Enter the Author of the Book:')
        self.newBook={'Book_Title':self.Title,
                      'Author_Name':self.Author,
                      '_id':int(self.ISBN),
                      'issue_date':None,
                      'renew_date':None,
                      'assigned_to':None,
                      'penalty':0}
        self.bookCollection.insert_one(self.newBook)   # Inserting the newly entered book to Library
        print(f'Book with ISBN {self.ISBN} is successfully added to Repository')
                
        self.final=self.thankYou()
        return self.final
        
    # This method is used to issue the book to Student
    def issueBook(self):
        self.bookCollection=mongo['library']['books']
        self.studentCollection=mongo['library']['student']
        
        # ISBN validation to assign to student
        self.issueISBN=int(input('Enter the ISBN of the Book to issue:'))
        self.issueBookCheck=self.bookCollection.find_one({'_id':self.issueISBN},{})
        
        if self.issueBookCheck == None:                   # Validation of Book 
            print(f'Book with ISBN {self.issueISBN} is not in Repository. Pls retry again.')
            self.issueBook()    
        elif self.issueBookCheck['assigned_to'] != None:   # Validation of Availaibility of Book
            print(f"Book with ISBN {self.issueISBN} is already assigned to {self.issueBookCheck['assigned_to']}")
            self.issueBook()
        else:
            # Student ID to assign the book
            self.issueID=int(input('Enter the Student ID to Issue Book :'))

            self.issueIDCheck=self.studentCollection.find_one({'_id':self.issueID},{})
            if self.issueIDCheck == None:       #validation of Student ID
                print(f'Student ID {self.issueID} is not in Reporsitory. Pls retry again.')
                self.issueBook()
            else:
                # Taking Issue date as Input.
                self.issueDate=input('Enter the issue date in YYYY-MM-DD format:')

                #Calculating the renew date. 
                self.interDate = datetime.strptime(self.issueDate, "%Y-%m-%d")
                self.renewDate = str(self.interDate + timedelta(days=15))
                self.renewDate = self.renewDate[0:10]

                #Assigning book to Student then updating Issue date and Renew date. 
                self.issueQuery={'_id':self.issueISBN}
                self.issueData={'$set':{'issue_date':self.issueDate,'assigned_to':self.issueID,'renew_date':self.renewDate}}
                self.bookCollection.update_one(self.issueQuery,self.issueData)
                print(f"\nBook {self.issueISBN} is assigned to {self.issueID}. Thank you!! ") 

                self.final=self.thankYou()
        return self.final

    # This method is for Editing the Books in the Library with new Title and Author name
    def editBook(self):
        self.bookCollection=mongo['library']['books']
        
        self.editISBN=int(input('\nEnter book ISBN to edit:'))  # ISBN validation
        self.editISBNCheck=self.bookCollection.find_one({'_id':self.editISBN},{})

        if self.editISBN == None:
            print(f'Book with ISBN {self.editISB} does not exist in Library. Pls retry with another book!!!!')
            self.editBook()
        else:
            self.newTitle=input('Enter the Title to update:')
            self.newAuthor=input('Enter the Author to update:')
            self.eQuery={'_id':self.editISBN}
            self.eUpdate={'$set':{'Book_Title':self.newTitle,'Author_Name':self.newAuthor}}
            self.bookCollection.update_one(self.eQuery,self.eUpdate)  # Updating the collection
            print(f"\nBook {self.editISBN} is updated with new values.Thank you!! ")
     
        self.final=self.thankYou()
        return self.final

    # This method is used to delete the Book entry from the Library. 
    def deleteBook(self):
        self.bookCollection=mongo['library']['books']
        self.deleteInput=input('Enter book ISBN or Title to delete:')
        
        if self.deleteInput.isdigit() == True:
            self.query={'_id':int(self.deleteInput)}
        else:
            self.query={'Book_Title':self.deleteInput}
            
        self.deletedData=self.bookCollection.delete_one(self.query,{}) #  Deleting the Entry
        if self.deletedData.deleted_count == 0:
            print(f'No Book for {self.deleteInput} in the repository')
        else:
            print(f'Book for {self.deleteInput} is sucessfully deleted')       
 
        self.final=self.thankYou()
        return self.final
                  
    # This method is used to Receive the book from Student. Accept the Book Return
    def receiveBook(self):
        self.bookCollection=mongo['library']['books']
        self.studentCollection=mongo['library']['student']
        
        self.receiveISBN=int(input('\nEnter book ISBN to receive:'))  # Accepting ISBN
        self.receiveCheck=self.bookCollection.find_one({'_id':self.receiveISBN},{'Author_Name':0,'_id':0})

        if self.receiveCheck == None:  # ISBN validation
            print(f'Book with ISBN {self.receiveISBN} does not exist in Library. Pls retry with another book!!!!')
            self.receiveBook()
        else:
            self.receiveID=int(input('Enter the Student ID to return book:')) #Accepting Student ID
            self.receiveIDCheck=self.studentCollection.find_one({'_id':self.receiveID},{'_id':1})
            if self.receiveIDCheck == None:
                print(f'Student with user ID {self.receiveID} does not exist. Pls retry')
                self.receiveBook()
            elif self.receiveCheck['assigned_to'] != self.receiveIDCheck['_id']:
                print(f'Book with ISBN {self.receiveISBN} is not assigned to Student with {self.receiveID}. Pls retry')
                self.receiveBook()
            else:
                self.daysDiff = self.renewDateValidation(self.receiveCheck['renew_date'])  #Calls function to validate Renew Date
                self.inpQuery={'_id':self.receiveISBN}
                self.updBook={'$set':{'issue_date':None,'renew_date':None,'assigned_to':None,'penalty':0}} #Initailize the fields
                
                if self.daysDiff > 0:
                    self.penalty = self.daysDiff * 20
                    print(f"\nYou have to accept Penalty of Rs.{self.penalty} from user {self.receiveID}!!!")
                    print('Enter 1, if user ready to pay penalty.\nEnter 2, if user not ready to pay Penalty now')
                    self.penaltyCheck=input()    # Validate the Penalty Payment Approval
                    
                    if int(self.penaltyCheck) == 1:
                        self.bookCollection.update_one(self.inpQuery,self.updBook) 
                        print(f'Thank you for updating entry for book {self.receiveISBN}.')
                        self.calcPenalty(self.penalty)      
                    elif int(self.penaltyCheck) == 2:
                        print('Thank You.. Pls Receive the book with Penalty again!')
                    else:
                        print('Invalid Option. Retry from Beginning')
                else:
                    self.bookCollection.update_one(self.inpQuery,self.updBook)    # Update the collection, if everything is FINE
                    print(f'Thank you for updating entry for book {self.receiveISBN}.')
          
        self.final=self.thankYou()
        return self.final
                  
    # This method is used to Renew the book, where expiry date is less than or equal to current date
    def renewBook(self):
        self.bookCollection=mongo['library']['books']
        
        self.renewISBN=int(input('\nEnter book ISBN to renew:'))  # Accepts ISBN
        self.renewCheck=self.bookCollection.find_one({'_id':self.renewISBN},{'Author_Name':0,'_id':0})

        if self.renewCheck == None:    #ISBN Validation
            print(f'Book with ISBN {self.receiveISBN} does not exist in Library. Pls retry with another book!!!!')
            self.renewBook()
        else:
            self.daysDiff = self.renewDateValidation(self.renewCheck['renew_date'])
            if self.daysDiff > 0:    # If renew date is expired, ISBN cannot be renewed. 
                print("Renew Date is expired. This book cannot be renewed. Pls receive this book with Penalty")
            else:
                 #Calculating the renew date i.e.,  15 days from existing renew date
                self.iDate = datetime.strptime(self.renewCheck['renew_date'], "%Y-%m-%d")
                self.newRenewDate = str(self.iDate + timedelta(days=15))
                self.newRenewDate = self.newRenewDate[0:10]
                      
                #Updating Renew date. 
                self.iQuery={'_id':self.renewISBN}
                self.uData={'$set':{'renew_date':self.newRenewDate}}
                self.bookCollection.update_one(self.iQuery,self.uData) 
                print(f'The book {self.renewISBN} is updated with new renew date {self.newRenewDate}')
    
                
        self.final=self.thankYou()
        return self.final
                  
    # This method used to display books those are already Issued to Students. 
    def issuedBooks(self):
        print('\n**************List of Issued Books in Library**************\n')
        bookCollection=mongo['library']['books']
        issuedBookList=bookCollection.find({'assigned_to':{'$ne':None}},{})  # Fetches all documents from collection where assigned_to is None
        availableBooksdf=pd.DataFrame(list(issuedBookList))
        print(availableBooksdf)
        
        self.final=self.thankYou()
        return self.final
                  
    # This method is used to update the Penalty Amount being collected. This maintains the total count of Penalty collected. 
    def calcPenalty(self,penAmt):
        self.penaltyCollection=mongo['library']['penalty']

        self.query={'_id':'penalty'}
        self.penaltyCheck=self.penaltyCollection.find_one(self.query,{'calculatedPenalty':1})  
        if self.penaltyCheck == None:   #Creates the Penalty collection if its not present
            self.penaltyCollection.insert_one({'_id':'penalty','calculatedPenalty':penAmt}) 
        else:    
        #self.penaltyCheck=self.penaltyCollection.find_one(self.query,{'calculatedPenalty':1}) 
            self.existingValue=self.penaltyCheck['calculatedPenalty']
            self.newValue=self.existingValue+penAmt     
            self.penaltyCollection.update_one(self.query,{'$set':{'calculatedPenalty':self.newValue}}) #Updates the collection with penalty amount
        
        self.penaltyCheck=self.penaltyCollection.find_one(self.query,{'calculatedPenalty':1}) 
        print('Total Penalty Collected so far:',self.penaltyCheck['calculatedPenalty'])
        
        self.final=self.thankYou()
        return self.final
                  
#This is the class for Student Options
class student(library):
    def __init__(self):
        print('\nHey Hi!! Welcome to the Student Library Management System')
        self.studentCollection=mongo['library']['student']
    
    # This is the method to Student to Login or Register
    def studentLogin(self):
        self.studentCollection=mongo['library']['student']
        self.option = input('\nHello Student!! \nEnter 1 to Login \nEnter 2 to Register!\n')
        if self.option.isdigit() and (int(self.option) == 1 or int(self.option) == 2):
            if int(self.option) == 1:   # Logging in with existing User ID
                self.loginID = int(input("Enter the User ID(Student's Mobile Number) to login:"))
                self.searchID=self.studentCollection.find_one({'_id':self.loginID},{})
                if self.searchID == None:                           #Validating UserID
                    print(f"User {self.loginID} not in Database. Pls try with valid UserID")
                    self.studentLogin()
                else:  
                    self.loginPWD = input("Enter the password to login:")
                    if self.searchID['password'] == self.loginPWD:  #Validating Password
                        print(f'\nHello {self.loginID}. You have successfully logged In!')
                        self.studentMenuChoice=self.studentMenu()
                    else:
                        print('Incorrect Password. Pls re-try')
                        self.studentLogin()
            elif int(self.option) == 2:    # New User ID
                self.newStudent={}
                self.newStudent['_id']=int(input("Enter the User's Mobile Number:"))
                self.searchData2=self.studentCollection.find_one({'_id':self.newStudent['_id']},{})
                if self.searchData2 != None:  # Validating UserID if is already present in database
                    print(f"\nUser ID {self.newStudent['_id']} already exist in database. Pls login with ID or Register with new ID")
                    self.studentLogin()    
                self.newStudent['name']=input("\nEnter the new User's Name     :")
                self.newStudent['password']=input('Enter the password            :')
                self.newStudent['roll_number']=input("Enter the User's Roll Number in DSGUVOnnnn format  :")
                self.newStudent['department']=input("Enter the User's Department   :")
                self.newStudent['mail_id']=input("Enter the User's Email Adress :")

                self.studentCollection.insert_one(self.newStudent)   # Inserting new User info tho the collection
                print(f"User ID {self.newStudent['_id']} is added succesfully")
                self.studentMenuChoice = 0
                self.loginID = self.newStudent['_id']
        else:
            print('You have enter Invalid Option. Pls retry!')
            self.studentLogin()
                      
        return self.studentMenuChoice,self.loginID     # Returning Student choice and Login ID. 
        
        self.final=self.thankYou()
        return self.final
                      
# This method is used to display Student Menu and accept user input                      
    def studentMenu(self):
        print('\nChoose from the below Menu Options\n*************************************!')  
        print('Enter 1 to SEARCH the Book using Title or Author Name.')
        print('Enter 2 to View list of ALL Books.')
        print('Enter 3 to SEARCH Available Books.')   
        print('Enter 4 to View List of ASSIGNED Books to you.')
        self.studentChoice=input()
                      
        if self.studentChoice.isdigit() == False or int(self.studentChoice) < 1 or int(self.studentChoice) > 4:
            print('Invalid Selection. Pls try again!!')  #Validating option selected
            self.studentMenu()
        else:
            self.studentChoice = int(self.studentChoice)
        
        return self.studentChoice
        
        self.final=self.thankYou()
        return self.final
                      
    # This is the funcctionality of Student to search book using Title or Author
    def searchBook(self):
        self.bookCollection=mongo['library']['books']
        self.searchInputChoice=input('\nEnter 1 to search on Title \nEnter 2 to search by Author Name:\n')
        
        if self.searchInputChoice.isdigit() == True and int(self.searchInputChoice) == 1:
            self.searchInput=input('Enter the Book Title to search:')
            self.query={'Book_Title':self.searchInput}
        elif self.searchInputChoice.isdigit() == True and int(self.searchInputChoice) == 2:
            self.searchInput=input('Enter the Author Name to search:')
            self.query={'Author_Name':self.searchInput}
        else:
            print('Invalid Option. Pls retry again')
            self.searchBook()
            
        self.searchData=self.bookCollection.find(self.query,{'penalty':0,'issue_date':0,'renew_date':0})
        if self.searchData == None:
            print(f'No Book for {self.searchInput} in the repository')
        else:
            print(f'\nDetails for {self.searchInput} is -')
            dataDF=pd.DataFrame(list(self.searchData))
            print(dataDF)
        
        self.final=self.thankYou()
        return self.final
                      
# This methd displays all the books assigned to particular user who has logged in                      
    def myBooksList(self,stuID):
        print('\n**************List of Books assigned to Me**************\n')
        self.bookCollection=mongo['library']['books']
        self.availableBookList=self.bookCollection.find({'assigned_to':stuID},{'assigned_to':0})
        #UPDATE THE PENALTY IN DB USING FOR LOOP. 
        self.availableBooksdf=pd.DataFrame(list(self.availableBookList))
        print(self.availableBooksdf)
                              
        self.final=self.thankYou()
        return self.final

In [3]:
# Creating the Object user with the class library
final='Y'
user=library()

while(final=='Y'):
    #Invoking the Main Menu for the user. 
    choice=user.mainMenu()

    #If the choice is 1, admin/Librarian functionalities are executable..
    if choice == 1:
        a=admin()                   # Creates Admin/Librarian's Object
        admChoice=a.adminLogin()
        a.updatePenalty()
        if admChoice==1:
            final=a.addBook()             # If the option selected is 1, then call addBook() method to add new book.
        elif admChoice==2:
            final=a.editBook()            # If the option selected is 2, then call editBook()  to update either Title/Author. 
        elif admChoice==3:
            final=a.deleteBook()          # If the option selected is 3, then call deleteBook()  to delete from Repository.
        elif admChoice==4:
            final=a.issueBook()           # If the option selected is 4, then call issueBook()  to issue book to Student.
        elif admChoice==5:     
            final=a.receiveBook()         # If the option selected is 5, then call receiveBook() to receive book from Student.
        elif admChoice==6:
            final=a.renewBook()           # If the option selected is 6, then call renewBook() to update new renew date.
        elif admChoice==7:
            final=a.issuedBooks()         # If the option selected is 7, then call issuedBooks() to display issued books.
        elif admChoice==8:
            final=a.availableBooksList()  # If the option selected is 8, then call availableBooksList() to display Unassigned Books.
        elif admChoice==9:
            final=final=a.allBooksList()        # If the option selected is 9, then call allBooksList() to display all books in Library.
        elif admChoice==10:         # If the option selected is 10, then call calcPenalty() to calculate Penalty.
            penaltyAmount=input('Enter the penalty amount collected:')
            if penaltyAmount.isdigit() == False:
                print('Please enter the Valid Amount and try again!!')
            else:
                penaltyAmount=int(penaltyAmount)
                final=a.calcPenalty(penaltyAmount)  
    #If the choice is 2, student functionalities are executable..
    else:
        s=student()                 # Creates Student's Object
        stuChoice,stuID=s.studentLogin()
        s.updatePenalty()
        if stuChoice==1:
            final=s.searchBook()          # If the option selected is 1, then call searchBook() to search on Title or Author.
        elif stuChoice==2:  
            final=s.allBooksList()        # If the option selected is 2, then call allBooksList()  to display all Books.
        elif stuChoice==3:
            final=s.availableBooksList()  # If the option selected is 3, then call availableBooksList() to display available Books.
        elif stuChoice==4:
            final=s.myBooksList(stuID)    # If the option selected is 4, then call myBooksList() to display books assigned to a Student. 

Enter 1 (if you are an Admin) or 2 (if you are Student):1

Hey Hi!! Welcome to the Library Management System
Hello Admin. Pls login to proceed

Enter the Login ID :admin
Enter the Password :admin

Congrats, you have logged in succesfully as Admin

Choose the Option Menu Option
*************************************!
Enter 1 to ADD New book into Repository.
Enter 2 to EDIT existing book in Repository.
Enter 3 to DELETE book from Repository.
Enter 4 to ISSUE the book to Student.
Enter 5 to RECEIVE the book.
Enter 6 to RENEW the book.
Enter 7 to view ISSUED Books.
Enter 8 to view list of AVAILABLE Books.
Enter 9 to view list of ALL Books.
Enter 10 to accept PENALTY.
7

**************List of Issued Books in Library**************

     _id                  Book_Title   Author_Name  issue_date  renew_date  \
0  12345           Harry Potter V1.1       Rowling  2022-10-01  2022-10-31   
1  12346                  Nityotsava  Nissar Ahmed  2022-10-03  2022-10-18   
2  12347                   The 